In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.5.0`
import $ivy.`org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.0`

import $ivy.`org.apache.kafka:kafka-clients:3.5.1`
import $ivy.`org.testcontainers:testcontainers:1.19.3`
import $ivy.`org.testcontainers:kafka:1.19.3`

import $ivy.$
import $ivy.$
import $ivy.$
import $ivy.$
import $ivy.$

In [5]:

import org.apache.spark.sql.SparkSession
import org.apache.kafka.clients.producer._

import org.testcontainers.containers.KafkaContainer
import org.testcontainers.utility.DockerImageName
import java.util.Properties

val spark = SparkSession.builder()
  .appName("ScalaSpark")
  .master("local[*]")
  .getOrCreate()

import spark.implicits._

val kafkaContainer = new KafkaContainer(
  DockerImageName.parse("confluentinc/cp-kafka:7.6.5")
)

kafkaContainer.start()

val bootstrapServers = kafkaContainer.getBootstrapServers
val kafkaProps = new Properties()
kafkaProps.put("bootstrap.servers", bootstrapServers)
kafkaProps.put("key.serializer", "org.apache.kafka.common.serialization.StringSerializer")
kafkaProps.put("value.serializer", "org.apache.kafka.common.serialization.StringSerializer")

val kafkaProducer = new KafkaProducer[String, String](kafkaProps)

def sendToKafka(topic: String, values: Seq[String]): Unit = {
 
  values.foreach { value =>
    val record = new ProducerRecord[String, String](topic, value)
    kafkaProducer.send(record)
  }
  kafkaProducer.flush()
}

val kafkaTopic = "my-topic-scala"


25/03/30 16:30:51 INFO 5: Creating container for image: confluentinc/cp-kafka:7.6.5
25/03/30 16:30:51 INFO 5: Container confluentinc/cp-kafka:7.6.5 is starting: 16ddc0d6a1f00a420518c1e79df70bae185aa88b6c9722bcf14f9e307f6a8c1c
25/03/30 16:30:56 INFO 5: Container confluentinc/cp-kafka:7.6.5 started in PT4.245822057S
25/03/30 16:30:56 INFO ProducerConfig: ProducerConfig values: 
	acks = -1
	auto.include.jmx.reporter = true
	batch.size = 16384
	bootstrap.servers = [PLAINTEXT://localhost:57629]
	buffer.memory = 33554432
	client.dns.lookup = use_all_dns_ips
	client.id = producer-2
	compression.type = none
	connections.max.idle.ms = 540000
	delivery.timeout.ms = 120000
	enable.idempotence = true
	interceptor.classes = []
	key.serializer = class org.apache.kafka.common.serialization.StringSerializer
	linger.ms = 0
	max.block.ms = 60000
	max.in.flight.requests.per.connection = 5
	max.request.size = 1048576
	metadata.max.age.ms = 300000
	metadata.max.idle.ms = 300000
	metric.reporters = []
	metr

import org.apache.spark.sql.SparkSession
import org.apache.kafka.clients.producer._
import org.testcontainers.containers.KafkaContainer
import org.testcontainers.utility.DockerImageName
import java.util.Properties
spark: SparkSession = org.apache.spark.sql.SparkSession@5daedaf9
import spark.implicits._
kafkaContainer: KafkaContainer = GenericContainer(extraHosts=[], image=RemoteDockerImage(imageName=confluentinc/cp-kafka:7.6.5, imagePullPolicy=DefaultPullPolicy(), imageNameSubstitutor=org.testcontainers.utility.ImageNameSubstitutor$LogWrappedImageNameSubstitutor@9440958), volumesFroms=[], linkedContainers={}, startupCheckStrategy=org.testcontainers.containers.startupcheck.IsRunningStartupCheckStrategy@6f4ead4f, startupAttempts=1, workingDirectory=null, shmSize=null, copyToFileContainerPathMap={}, copyToTransferableContainerPathMap={}, dependencies=[], dockerClient=LazyDockerClient, containerId=16ddc0d6a1f00a420518c1e79df70bae185aa88b6c9722bcf14f9e307f6a8c1c, containerInfo=InspectContai

In [6]:
val data = Seq("a", "b", "c", "d", "e")

sendToKafka(kafkaTopic, data)

25/03/30 16:30:59 WARN NetworkClient: [Producer clientId=producer-2] Error while fetching metadata with correlation id 5 : {my-topic-scala=LEADER_NOT_AVAILABLE}
25/03/30 16:30:59 WARN NetworkClient: [Producer clientId=producer-2] Error while fetching metadata with correlation id 6 : {my-topic-scala=LEADER_NOT_AVAILABLE}
25/03/30 16:30:59 INFO Metadata: [Producer clientId=producer-2] Resetting the last seen epoch of partition my-topic-scala-0 to 0 since the associated topicId changed from null to LG7PdRV8Q_SUxsS2cAKRig


data: Seq[String] = List("a", "b", "c", "d", "e")

In [7]:
val df = spark.read
  .format("kafka")
  .option("kafka.bootstrap.servers", bootstrapServers)
  .option("subscribe", kafkaTopic)
  .load()
  .selectExpr("CAST(value AS STRING) as message")

  df.show()

25/03/30 16:31:45 INFO CodeGenerator: Code generated in 9.704422 ms
25/03/30 16:31:45 INFO CodeGenerator: Code generated in 18.34098 ms
25/03/30 16:31:46 INFO AdminClientConfig: AdminClientConfig values: 
	auto.include.jmx.reporter = true
	bootstrap.servers = [PLAINTEXT://localhost:57629]
	client.dns.lookup = use_all_dns_ips
	client.id = 
	connections.max.idle.ms = 300000
	default.api.timeout.ms = 60000
	metadata.max.age.ms = 300000
	metric.reporters = []
	metrics.num.samples = 2
	metrics.recording.level = INFO
	metrics.sample.window.ms = 30000
	receive.buffer.bytes = 65536
	reconnect.backoff.max.ms = 1000
	reconnect.backoff.ms = 50
	request.timeout.ms = 30000
	retries = 2147483647
	retry.backoff.ms = 100
	sasl.client.callback.handler.class = null
	sasl.jaas.config = null
	sasl.kerberos.kinit.cmd = /usr/bin/kinit
	sasl.kerberos.min.time.before.relogin = 60000
	sasl.kerberos.service.name = null
	sasl.kerberos.ticket.renew.jitter = 0.05
	sasl.kerberos.ticket.renew.window.factor = 0.8
	sa

+-------+
|message|
+-------+
|      a|
|      b|
|      c|
|      d|
|      e|
+-------+



df: org.apache.spark.sql.package.DataFrame = [message: string]